# Exercise05 : Distributed Training with Curated Environments

Here we change our sample (see "[Exercise03 : Just Train in Your Working Machine](./exercise03_train_simple.ipynb)") for distributed training using multiple machines in Azure Machine Learning.

In this exercise, we run Horovod framework in this pre-configured environment. (As you saw in previous [Exercise04](./exercise04_train_remote.ipynb), you can also configure distributed training with manually-configured custom environment by ```azureml.core.ScriptRunConfig```.)

*back to [index](https://github.com/tsmatz/azureml-tutorial/)*

## Save your training script as file (train.py)

Create ```scirpt``` directory.

In [1]:
import os
script_folder = './script'
os.makedirs(script_folder, exist_ok=True)

Change our original source code ```train.py``` (see "[Exercise03 : Just Train in Your Working Machine](./exercise03_train_simple.ipynb)") as follows. The lines commented "##### modified" is modified lines.    
After that, please add the following ```%%writefile``` at the beginning of the source code and run this cell.    
This source code will then be saved as ```./script/train_horovod.py```.

In [2]:
%%writefile script/train_horovod.py
import os
import argparse
import tensorflow as tf

import horovod.tensorflow.keras as hvd ##### modified

# device test
print("##### List of available GPU #####")
print(tf.config.list_physical_devices("GPU"))

# parse arguments
parser = argparse.ArgumentParser()
parser.add_argument(
    "--data_folder",
    type=str,
    default="./data",
    help="Folder path for input data")
parser.add_argument(
    "--model_folder",
    type=str,
    default="./outputs",  # AML experiments outputs folder
    help="Folder path for model output")
parser.add_argument(
    "--learning_rate",
    type=float,
    default="0.001",
    help="Learning Rate")
parser.add_argument(
    "--first_layer",
    type=int,
    default="128",
    help="Neuron number for the first hidden layer")
parser.add_argument(
    "--second_layer",
    type=int,
    default="64",
    help="Neuron number for the second hidden layer")
parser.add_argument(
    "--epochs_num",
    type=int,
    default="6",
    help="Number of epochs")
FLAGS, unparsed = parser.parse_known_args()

hvd.init() ##### modified

# Horovod config output
print("##### Horovod config #####")
print("Size {}".format(hvd.size()))
print("Rank {}".format(hvd.rank()))

# build model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(FLAGS.first_layer, activation="relu"),
    tf.keras.layers.Dense(FLAGS.second_layer, activation="relu"),
    tf.keras.layers.Dense(10)
])
opt = tf.keras.optimizers.Adam(FLAGS.learning_rate)
opt = hvd.DistributedOptimizer(opt) ##### modified
model.compile(
    optimizer=opt,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

# run training
train_data_path = os.path.join(FLAGS.data_folder, "train")
train_data = tf.data.experimental.load(train_data_path)
model.fit(
    train_data.shuffle(1000).batch(128).prefetch(tf.data.AUTOTUNE),
    callbacks=[hvd.callbacks.BroadcastGlobalVariablesCallback(0)],  ##### modified
    epochs=FLAGS.epochs_num
)

# save model and variables
if hvd.rank() == 0 : ##### modified
    model_path = os.path.join(FLAGS.model_folder, "mnist_tf_model")
    model.save(model_path)
    print("current working directory : ", os.getcwd())
    print("model folder : ", model_path)

Writing script/train_horovod.py


## Train on multiple machines (Horovod)

### Step 1 : Get workspace setting

Before starting, you must read your configuration settings. (See "[Exercise01 : Prepare Config Settings](./exercise01_prepare_config.ipynb)".)

In [3]:
from azureml.core import Workspace
import azureml.core

ws = Workspace.from_config()

### Step 2 : Create multiple virtual machines (cluster)

Create your new AML compute for distributed clusters. By enabling auto-scaling from 0 to 4, you can save money (all nodes are terminated) if it's inactive.<br>
If already exists, this script will get the existing cluster.

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name='mycluster01')
    print('found existing:', compute_target.name)
except ComputeTargetException:
    print('creating new.')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_D2_v2',
        min_nodes=0,
        max_nodes=3)
    compute_target = ComputeTarget.create(ws, 'mycluster01', compute_config)
    compute_target.wait_for_completion(show_output=True)

creating new.
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Step 3 : Get dataset reference for files

You can mount your registered dataset (See "[Exercise02 : Prepare Data](./exercise02_prepare_data.ipynb)") into your AML compute.<br>
Now we get the registered dataset reference.

In [5]:
from azureml.core import Dataset

dataset = Dataset.get_by_name(ws, 'mnist_dataset', version='latest')

# # For using unregistered data, see below
# from azureml.core import Datastore
# from azureml.core import Dataset
# ds = ws.get_default_datastore()
# ds_paths = [(ds, 'tfdata/')]
# dataset = Dataset.File.from_files(path = ds_paths)

### Step 4 : Configure with Curated Environment

Here we run distributed training by Horovod with TensorFlow.<br>
In this training, this job will be distributed on 3 node.

In this example, I also use the registered data asset named ```mnist_dataset``` to mount in your compute target. (Run "[Exercise02 : Prepare Data](./exercise02_prepare_data.ipynb)" for data preparation.)

> Note : In this example, I have used built-in GPU environment (```AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu```) on CPU cluster. If GPU is not available, it will correctly run on CPU.<br>
> When you prefer CPU image, you can also create and configure your own image. (See [Exercise04](./exercise04_train_remote.ipynb).)

In [6]:
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import MpiConfiguration

tf_env = Environment.get(workspace=ws, name='AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu')
src = ScriptRunConfig(
    source_directory='./script',
    script='train_horovod.py',
    arguments=['--data_folder', dataset.as_mount()],
    compute_target=compute_target,
    environment=tf_env,
    distributed_job_config=MpiConfiguration(node_count=3))

[Optional] This ```AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu``` includes Horovod 0.23.0.    
When you want to see the packages included in this curated environment, please run as follows and see the saved configuration.

In [7]:
tf_env.save_to_directory(path='AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu')

### Step 5 : Run script and wait for completion

In [8]:
from azureml.core import Experiment

exp = Experiment(workspace=ws, name='tf_distribued')
run = exp.submit(config=src)
run.wait_for_completion(show_output=True)

RunId: tf_distribued_1664947621_5ffe52c6
Web View: https://ml.azure.com/runs/tf_distribued_1664947621_5ffe52c6?wsid=/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourcegroups/rg-AML/workspaces/ws01&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/55_azureml-execution-tvmps_12fbfd615042dbfa79b76d1c978505c7b3bf8a4efe7d4c03819efe819a61038a_d.txt

2022-10-05T05:34:26Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/ws01/azureml/tf_distribued_1664947621_5ffe52c6/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/ws01/azureml/tf_distribued_1664947621_5ffe52c6/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=92042 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_root/jobs/ws01/azureml/tf_distribued_1664947621_5ffe52c6/configs/workspaceblobstore.cfg --log-level=LOG_WARNING
2022-10-05T05:34:27Z Successfully mounted a/an Blobfuse File System at /mnt/


Streaming azureml-logs/70_driver_log_0.txt

bash: /azureml-envs/tensorflow-2.7/lib/libtinfo.so.6: no version information available (required by bash)
[2022-10-05T05:53:39.746223] Entering context manager injector.
[2022-10-05T05:53:41.200823] context_manager_injector.py Command line Options: Namespace(inject=['ProjectPythonPath:context_managers.ProjectPythonPath', 'Dataset:context_managers.Datasets', 'RunHistory:context_managers.RunHistory', 'TrackUserError:context_managers.TrackUserError', 'UserExceptions:context_managers.UserExceptions'], invocation=['train_horovod.py', '--data_folder', 'DatasetConsumptionConfig:input__16c18986'])
This is an MPI job. Rank:0
Script type = None
[2022-10-05T05:53:41.210888] Entering Run History Context Manager.
[2022-10-05T05:53:43.984626] Current directory: /mnt/batch/tasks/shared/LS_root/jobs/ws01/azureml/tf_distribued_1664947621_5ffe52c6/wd/azureml/tf_distribued_1664947621_5ffe52c6
[2022-10-05T05:53:43.984685] Preparing to call script [train_horovod


Streaming azureml-logs/75_job_post-tvmps_12fbfd615042dbfa79b76d1c978505c7b3bf8a4efe7d4c03819efe819a61038a_d.txt

[2022-10-05T05:54:53.123257] Entering job release
[2022-10-05T05:54:54.927912] Starting job release
[2022-10-05T05:54:54.928876] Logging experiment finalizing status in history service.[2022-10-05T05:54:54.929160] job release stage : upload_datastore starting...

Starting the daemon thread to refresh tokens in background for process with pid = 376
[2022-10-05T05:54:54.929999] job release stage : start importing azureml.history._tracking in run_history_release.
[2022-10-05T05:54:54.931980] job release stage : execute_job_release starting...
[2022-10-05T05:54:54.940306] job release stage : copy_batchai_cached_logs starting...
[2022-10-05T05:54:54.941598] job release stage : copy_batchai_cached_logs completed...
[2022-10-05T05:54:54.950885] Entering context manager injector.
[2022-10-05T05:54:54.953413] job release stage : upload_datastore completed...
[2022-10-05T05:54:55.017

{'runId': 'tf_distribued_1664947621_5ffe52c6',
 'target': 'mycluster01',
 'status': 'Completed',
 'startTimeUtc': '2022-10-05T05:34:21.320567Z',
 'endTimeUtc': '2022-10-05T05:55:09.124626Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '854efebe-1b17-4b1c-bd49-a1410fb8e940',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '16c18986-c760-49b0-a222-eeb89a5f9262'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'input__16c18986', 'mechanism': 'Mount'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'train_horovod.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--data_folder', 'DatasetConsumptionConfig:input__16c18986'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'Mpi',
  'target': 'mycluster01',
  'dataReferences': {},
  'data': {'input__16c18986': {'dataLocation

### Step 6 : Download results and evaluate

In [9]:
run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_12fbfd615042dbfa79b76d1c978505c7b3bf8a4efe7d4c03819efe819a61038a_d.txt',
 'azureml-logs/55_azureml-execution-tvmps_3470ea1dc426ec1edc63dc1e5a809d35333ab87da750aa5e46f66c62b174f42c_d.txt',
 'azureml-logs/55_azureml-execution-tvmps_62518c13ed547277fc5c01d2938fcdf4a619468bef9bc755b56d84ea89f3c9a5_d.txt',
 'azureml-logs/65_job_prep-tvmps_12fbfd615042dbfa79b76d1c978505c7b3bf8a4efe7d4c03819efe819a61038a_d.txt',
 'azureml-logs/65_job_prep-tvmps_3470ea1dc426ec1edc63dc1e5a809d35333ab87da750aa5e46f66c62b174f42c_d.txt',
 'azureml-logs/65_job_prep-tvmps_62518c13ed547277fc5c01d2938fcdf4a619468bef9bc755b56d84ea89f3c9a5_d.txt',
 'azureml-logs/70_driver_log_0.txt',
 'azureml-logs/70_driver_log_1.txt',
 'azureml-logs/70_driver_log_2.txt',
 'azureml-logs/70_mpi_log.txt',
 'azureml-logs/75_job_post-tvmps_12fbfd615042dbfa79b76d1c978505c7b3bf8a4efe7d4c03819efe819a61038a_d.txt',
 'azureml-logs/75_job_post-tvmps_3470ea1dc426ec1edc63dc1e5a809d35333ab87da750aa5e46f66c6

In [10]:
run.download_file(
    name='outputs/mnist_tf_model/keras_metadata.pb',
    output_file_path='distributed_model/keras_metadata.pb')
run.download_file(
    name='outputs/mnist_tf_model/saved_model.pb',
    output_file_path='distributed_model/saved_model.pb')
run.download_file(
    name='outputs/mnist_tf_model/variables/variables.data-00000-of-00001',
    output_file_path='distributed_model/variables/variables.data-00000-of-00001')
run.download_file(
    name='outputs/mnist_tf_model/variables/variables.index',
    output_file_path='distributed_model/variables/variables.index')

In [11]:
import tensorflow as tf

test_data = tf.data.Dataset.load("./data/test")

loaded_model = tf.keras.models.load_model("./distributed_model")
for image, true_value in test_data.take(3):
    pred_output = loaded_model(tf.expand_dims(image, axis=0))
    pred_value = tf.math.argmax(pred_output, axis=-1).numpy().item()
    print("Predicted {}, True {}".format(pred_value, true_value))

2022-10-05 05:57:07.177595: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-05 05:57:07.330842: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-05 05:57:07.330879: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-05 05:57:07.364256: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-05 05:57:08.130785: W tensorflow/stream_executor/pla

Predicted 7, True 7
Predicted 2, True 2
Predicted 1, True 1


### Step 7 : Remove AML compute

**You don't need to remove your AML compute** for saving money, because the nodes will be automatically terminated, when it's inactive.    
But if you want to clean up, please run the following.

In [13]:
# Delete cluster (nbodes) and remove from AML workspace
mycompute = AmlCompute(workspace=ws, name='mycluster01')
mycompute.delete()